In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("scater")
# install.packages('Seurat')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.2 (2023-10-31)

Installing package(s) 'BiocVersion', 'scater'

also installing the dependencies ‘zlibbioc’, ‘bitops’, ‘XVector’, ‘sparseMatrixStats’, ‘RCurl’, ‘GenomeInfoDbData’, ‘abind’, ‘lambda.r’, ‘futile.options’, ‘sitmo’, ‘GenomicRanges’, ‘DelayedMatrixStats’, ‘Biobase’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘SparseArray’, ‘matrixStats’, ‘RcppHNSW’, ‘ScaledMatrix’, ‘irlba’, ‘rsvd’, ‘futile.logger’, ‘snow’, ‘BH’, ‘beeswarm’, ‘vipor’, ‘gridExtra’, ‘FNN’, ‘RcppAnnoy’, ‘RcppProgress’, ‘dqrng’, ‘Cairo’, ‘png’, ‘SingleCellExperiment’, ‘scuttle’, ‘BiocGenerics’, ‘S4Vectors’, ‘SummarizedExperiment’, ‘DelayedArray’, ‘MatrixGenerics’, ‘beachmat’, ‘BiocNeighbors’, ‘

In [ ]:
# if (!requireNamespace("remotes", quietly = TRUE)) {
#   install.packages("remotes")
# }
# remotes::install_github("mojaveazure/seurat-disk")

In [46]:
# install.packages('aricode')

In [79]:
library(scater)
library(Seurat)
library(SeuratDisk)
# library(SeuratData)
library(patchwork)
library(aricode)
library(tidyverse)

In [80]:
looms <- c('./hvg_objs/allen_b08_300hvgs.loom',
 './hvg_objs/cl5_2000hvgs.loom',
 './hvg_objs/allen_b02h01_300hvgs.loom',
 './hvg_objs/cl5_300hvgs.loom',
 './hvg_objs/allen_b08_2000hvgs.loom',
 './hvg_objs/allen_b02h01_1000hvgs.loom',
 './hvg_objs/allen_b08_4000hvgs.loom',
 './hvg_objs/cl3_1000hvgs.loom',
 './hvg_objs/allen_b08_1000hvgs.loom',
 './hvg_objs/cl3_2000hvgs.loom',
 './hvg_objs/cl5_1000hvgs.loom',
 './hvg_objs/cl5_4000hvgs.loom',
 './hvg_objs/allen_b02h01_4000hvgs.loom',
 './hvg_objs/cl3_4000hvgs.loom',
 './hvg_objs/allen_b02h01_2000hvgs.loom',
 './hvg_objs/cl3_300hvgs.loom')

#'allen_b08', 'allen_b02h01', 'cl3', 'cl5', 'brca1', 'pbmc','e11e13'
shorts <- c('allen_b08','cl5','allen_b02h01','cl5','allen_b08','allen_b02h01',
           'allen_b08','cl3','allen_b08','cl3','cl5','cl5','allen_b02h01','cl3','allen_b02h01','cl3')

In [81]:
length(looms)

[1] 16

In [82]:
res <- c(0.75,1,1.5,2)
xy <- vector("list", length(res)*length(shorts)) # create an empty list into which values are to be filled

for (o in 1:length(looms)){
    a08 <- Connect(filename = looms[o], mode = "r")
    
    
    # gene list
    n_genes <- a08[['row_attrs']][['gene_name']][['dims']]
    gns <- a08[['row_attrs']][['gene_name']][1:n_genes]
    
    # cell ID list
    n_cells <- a08[['col_attrs']][['barcode']][['dims']]
    cellids <- a08[['col_attrs']][['barcode']][1:n_cells]
    celllabs <- a08[['col_attrs']][['subclass_label']][1:n_cells]
    
    # get raw counts matrix
    raw.cnts <- t(a08[["layers/unspliced"]][,])
    colnames(raw.cnts) <- cellids
    # rownames(raw.cnts) <- gns
    
    
    metadata <- data.frame(
        cellID = cellids
        )
    rownames(metadata) <- cellids
    
    
    
    s_obj <- CreateSeuratObject(counts = raw.cnts,
                                project = "fromLoom",
                                assay = "URNA",
                                meta.data = metadata)
    
    sraw.cnts <- t(a08[["layers/spliced"]][,])
    colnames(sraw.cnts) <- cellids
    
    s_assay <- CreateAssay5Object(counts = sraw.cnts)
    
    s_obj[["SRNA"]] <- s_assay
    
    a08$close_all()
    
    DefaultAssay(s_obj) <- 'URNA'
    s_obj <- NormalizeData(s_obj) 
    # s_obj <- FindVariableFeatures(s_obj) 
    s_obj <- ScaleData(s_obj) 
    s_obj <- RunPCA(s_obj,reduction.name = 'upca', features = Features(s_obj))
    
    DefaultAssay(s_obj) <- 'SRNA'
    s_obj <- NormalizeData(s_obj) 
    # s_obj <- FindVariableFeatures(s_obj)
    s_obj <- ScaleData(s_obj)
    s_obj <- RunPCA(s_obj,reduction.name = 'spca',features = Features(s_obj))
    
    
    s_obj <- FindMultiModalNeighbors(
      s_obj, reduction.list = list("upca", "spca"), 
      dims.list = list(1:30, 1:18), modality.weight.name = "RNA.weight"
    )
    
    #Run Leiden here + save results
    num_features <- nrow(s_obj)
    
    
    for (i in 1:length(res)) {
      #Get leiden clusters for different res
      s_obj <- FindClusters(s_obj, graph.name = "wsnn", algorithm = 4, resolution = res[i], verbose = FALSE, cluster.name='leiden')
      ind <- (o-1)*length(res) + i 
      xy[[ind]] <- data.frame(Method = 'WNN-Leiden',
                            Clustering = I(list(s_obj[['leiden']]$leiden)),
                            Matrices = 'U,S',
                            Hyperparam = res[i],
                            HVGs = num_features ,
                            Data = shorts[o],
                            ARI = ARI(celllabs,s_obj[['leiden']]$leiden),
                            AMI = AMI(celllabs,s_obj[['leiden']]$leiden))
    }

}

xy <- do.call(rbind, xy)

xy %>% 
  rowwise() %>% 
  mutate_if(is.list, ~paste(unlist(.), collapse = '|')) %>% 
  write.csv('./fits/Seuratruns_0209/WNN_bench_results.csv', row.names = FALSE)

head(xy)
#shorts[o]

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
Normalizing layer: counts

Centering and scaling data matrix

Warning message:
“The following 20 features requested have zero variance; running reduction without them: Feature8, Feature39, Feature61, Feature75, Feature84, Feature100, Feature111, Feature147, Feature148, Feature149, Feature152, Feature162, Feature178, Feature186, Feature187, Feature243, Feature256, Feature276, Feature283, Feature287”
PC_ 1 
Positive:  Feature257, Feature119, Feature230, Feature17, Feature87, Feature133, Feature228, Feature216, Feature10, Feature114 
	   Feature235, Feature196, Feature188, Feature139, Feature42, Feature248, Feature290, Feature183, Feature140, Feature170 
	   Feature165, Feature143, Feature210, Feature131, Feature35, Feature62, Feature6, Feature83, Feature240, Feature289 
Negative:  Feature127, Feature234, Feature146, Feature32, Feature223, Feature28, Featur

,Method,Clustering,Matrices,Hyperparam,HVGs,Data,ARI,AMI
,<chr>,<I<list>>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,WNN-Leiden,"3, 2, 10....","U,S",0.75,299,allen_b08,0.5009574,0.6682406
2,WNN-Leiden,"3, 1, 10....","U,S",1.00,299,allen_b08,0.4414347,0.6338954
3,WNN-Leiden,"2, 1, 12....","U,S",1.50,299,allen_b08,0.4293264,0.6214793
4,WNN-Leiden,"14, 2, 1....","U,S",2.00,299,allen_b08,0.3090938,0.5502668
5,WNN-Leiden,"2, 2, 3,....","U,S",0.75,2000,cl5,0.4238502,0.5137069
6,WNN-Leiden,"2, 2, 3,....","U,S",1.00,2000,cl5,0.4282119,0.5095983
